In [1]:
import sklearn
import pandas as pd
import numpy as np

In [7]:
#import data
tweet_df=pd.read_csv("tweets.csv")

In [6]:
tweet_df


,Tweet,Intention
0,Ligi ya Championship: Msimamo wa jedwali.\n#Ka...,1
1,Inconvenience for the One Who Guaranteed Eve M...,1
2,Ligi ya DStv Premiership: Matokeo ya mechi.\n#...,1
3,Here Are the Names of the Previous US Presiden...,0
4,@TiborPNagyJr Uhuru Kenyatta is probably not p...,0
...,...,...
4568,"P R O M O !\n\nInitial Prices: 3,500 - 4000\nP...",1
4569,10T Toilet Paper Making Line Turnkey Project\n...,1
4570,Protonitazene CAS119276-01-6 powder\nGOOD EFFE...,1
4571,New deal alert 🚨 \nRegistered Toyota Camry \nB...,1


In [8]:
#check if there are any missing values
tweet_df.isnull().sum()

Tweet        0
Intention    0
dtype: int64

In [10]:
sum(tweet_df["Intention"] ==0)

2283

In [11]:
sum(tweet_df["Intention"]==1)

2290

In [12]:
tweet_df.head()

,Tweet,Intention
0,Ligi ya Championship: Msimamo wa jedwali.\n#Ka...,1
1,Inconvenience for the One Who Guaranteed Eve M...,1
2,Ligi ya DStv Premiership: Matokeo ya mechi.\n#...,1
3,Here Are the Names of the Previous US Presiden...,0
4,@TiborPNagyJr Uhuru Kenyatta is probably not p...,0


In [13]:
tweet_df.tail()

,Tweet,Intention
4568,"P R O M O !\n\nInitial Prices: 3,500 - 4000\nP...",1
4569,10T Toilet Paper Making Line Turnkey Project\n...,1
4570,Protonitazene CAS119276-01-6 powder\nGOOD EFFE...,1
4571,New deal alert 🚨 \nRegistered Toyota Camry \nB...,1
4572,Hi dear friends :\nour promotion has started \...,1


In [17]:
#Data Cleaning
!pip install tweet-preprocessor


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import re
#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [43]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [44]:
tweet_df_tweet=clean_tweets(tweet_df["Tweet"])

In [45]:
tweet_df_tweet=pd.DataFrame(tweet_df_tweet)

In [46]:
tweet_df["clean_tweet"]=tweet_df_tweet

In [47]:
tweet_df.head(10)

,Tweet,Intention,clean_tweet
0,Ligi ya Championship: Msimamo wa jedwali.\n#Ka...,1,ligi ya championship msimamo wa jedwali dj gra...
1,Inconvenience for the One Who Guaranteed Eve M...,1,inconvenience for the one who guaranteed eve m...
2,Ligi ya DStv Premiership: Matokeo ya mechi.\n#...,1,ligi ya dstv premiership matokeo ya mechi grau...
3,Here Are the Names of the Previous US Presiden...,0,here are the names of the previous us presiden...
4,@TiborPNagyJr Uhuru Kenyatta is probably not p...,0,uhuru kenyatta is probably not paid yet by and...
5,"Kenya's former president Uhuru Kenyatta, a key...",0,kenyas former president uhuru kenyatta a key p...
6,@CaxstonePKigata I should think Hon Mungatana ...,0,i should think hon mungatana was in wash wash ...
7,My Wife And Children Have Left Me - Ex-tahidi ...,1,my wife and children have left me ex tahidi ...
8,The AU talks was expected to be be mediated by...,0,the au talks was expected to be be mediated by...
9,"How to Unlock M-Kopa Phones Use Samsung, Nokia...",1,how to unlock m kopa phones use samsung nokia ...


In [48]:
from sklearn.model_selection import train_test_split

In [50]:
# extract the labels from the train data
y = tweet_df.Intention.values

In [52]:
# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(tweet_df.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

In [59]:
# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))


CountVectorizer(binary=True, stop_words='english')

In [60]:
# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [61]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [63]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  96.93877551020408 %


In [64]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [70]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       685
           1       0.98      0.95      0.97       687

    accuracy                           0.97      1372
   macro avg       0.97      0.97      0.97      1372
weighted avg       0.97      0.97      0.97      1372

